In [8]:
import os
os.makedirs("models", exist_ok=True)

import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from sklearn.utils.class_weight import compute_class_weight
import pandas as pd

# ←←← THIS IS THE MACBOOK FIX ←←←
from keras.optimizers.legacy import Adam, RMSprop, Adagrad, SGD   # ← legacy = FAST on Mac!
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, BatchNormalization, GlobalAveragePooling2D, Input
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator

from utils import load_lung_dataset, IMG_SIZE

In [9]:
# Load data
print("Loading dataset...")
X, y, class_names = load_lung_dataset()
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

Loading dataset...
Loading 5-class lung sound dataset...
→ asthma: 288 files


asthma: 100%|██████████| 288/288 [00:01<00:00, 197.18it/s]


→ Bronchial: 104 files


Bronchial: 100%|██████████| 104/104 [00:00<00:00, 196.19it/s]


→ copd: 401 files


...

→ healthy: 133 files


healthy: 100%|██████████| 133/133 [00:00<00:00, 189.34it/s]


→ pneumonia: 285 files


pneumonia: 100%|██████████| 285/285 [00:01<00:00, 198.74it/s]


Dataset loaded: 1211 samples
Shape: (1211, 128, 258, 1)


In [10]:
# Fix imbalance
class_weights = compute_class_weight('balanced', classes=np.arange(5), y=np.argmax(y_train, axis=1))
class_weight_dict = dict(enumerate(class_weights))

In [11]:
# Strong augmentation
datagen = ImageDataGenerator(
    rotation_range=20, width_shift_range=0.2, height_shift_range=0.2,
    zoom_range=0.3, horizontal_flip=True, fill_mode='reflect'
)

In [12]:
# Deep model
def build_model():
    inputs = Input(shape=(128, 258, 1))
    x = Conv2D(32, (3,3), activation='relu', padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Conv2D(32, (3,3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(2,2)(x)

    x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(2,2)(x)

    x = Conv2D(128, (3,3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(128, (3,3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(2,2)(x)

    x = Conv2D(256, (3,3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.4)(x)
    outputs = Dense(5, activation='softmax')(x)
    return Model(inputs, outputs)

In [13]:
optimizers = {
    'Adam': Adam(learning_rate=0.001),
    'RMSprop': RMSprop(learning_rate=0.001),
    'Adagrad': Adagrad(learning_rate=0.01),
    'SGD': SGD(learning_rate=0.01, momentum=0.9)
}

losses = ['categorical_crossentropy', 'mse']
results = []

print("\nSTARTING TRAINING — YOU WILL SEE PROGRESS BAR NOW!\n" + "="*80)

for loss_name in losses:
    print(f"\nLOSS: {loss_name.upper()}")
    print("-" * 70)
    
    for opt_name, opt in optimizers.items():
        print(f"\n→ Training with {opt_name}...", flush=True)
        
        model = build_model()
        model.compile(optimizer=opt, loss=loss_name, metrics=['accuracy'])
        
        history = model.fit(
            datagen.flow(X_train, y_train, batch_size=32),
            validation_data=(X_val, y_val),
            epochs=80,                  # reduced for speed
            verbose=1,                  # ← NOW YOU SEE PROGRESS BAR!
            callbacks=[
                ModelCheckpoint("models/temp.keras", save_best_only=True, monitor='val_accuracy', mode='max', verbose=0),
                EarlyStopping(patience=15, restore_best_weights=True, monitor='val_accuracy')
            ],
            class_weight=class_weight_dict if loss_name == 'categorical_crossentropy' else None
        )
        
        val_acc = max(history.history['val_accuracy'])
        y_pred = np.argmax(model.predict(X_val, verbose=0), axis=1)
        y_true = np.argmax(y_val, axis=1)
        p, r, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
        
        results.append({
            'Loss': loss_name,
            'Optimizer': opt_name,
            'Precision': round(p, 4),
            'Recall': round(r, 4),
            'F1-score': round(f1, 4),
            'Train_Acc': round(history.history['accuracy'][-1], 4),
            'Val_Acc': round(val_acc, 4)
        })
        print(f"Best Val Accuracy: {val_acc:.4f}")


STARTING TRAINING — YOU WILL SEE PROGRESS BAR NOW!

LOSS: CATEGORICAL_CROSSENTROPY
----------------------------------------------------------------------

→ Training with Adam...
Epoch 1/80
31/31 [==============================] - 20s 645ms/step - loss: 1.5853 - accuracy: 0.3709 - val_loss: 1.7695 - val_accuracy: 0.3292
Epoch 2/80
31/31 [==============================] - 20s 656ms/step - loss: 1.4743 - accuracy: 0.3988 - val_loss: 2.4583 - val_accuracy: 0.3292
Epoch 3/80
31/31 [==============================] - 21s 661ms/step - loss: 1.4559 - accuracy: 0.4452 - val_loss: 4.4559 - val_accuracy: 0.3292
Epoch 4/80
31/31 [==============================] - 20s 657ms/step - loss: 1.4002 - accuracy: 0.4669 - val_loss: 5.2198 - val_accuracy: 0.3292
Epoch 5/80
31/31 [==============================] - 21s 672ms/step - loss: 1.3818 - accuracy: 0.4680 - val_loss: 8.5537 - val_accuracy: 0.3292
Epoch 6/80
31/31 [==============================] - 21s 688ms/step - loss: 1.3845 - accuracy: 0.4504 - va

/Users/shalini/Documents/machine_learning_models/audio_classification_projects/myenv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


31/31 [==============================] - 20s 655ms/step - loss: 1.5967 - accuracy: 0.3006 - val_loss: 1.5873 - val_accuracy: 0.2387
Epoch 2/80
31/31 [==============================] - 20s 652ms/step - loss: 1.4854 - accuracy: 0.3585 - val_loss: 1.6624 - val_accuracy: 0.0864
Epoch 3/80
31/31 [==============================] - 20s 651ms/step - loss: 1.4577 - accuracy: 0.4122 - val_loss: 1.9395 - val_accuracy: 0.0864
Epoch 4/80
31/31 [==============================] - 20s 653ms/step - loss: 1.4311 - accuracy: 0.4225 - val_loss: 2.2324 - val_accuracy: 0.3045
Epoch 5/80
31/31 [==============================] - 20s 652ms/step - loss: 1.3422 - accuracy: 0.4556 - val_loss: 2.3803 - val_accuracy: 0.3292
Epoch 6/80
31/31 [==============================] - 20s 653ms/step - loss: 1.3244 - accuracy: 0.4638 - val_loss: 2.6560 - val_accuracy: 0.3210
Epoch 7/80
31/31 [==============================] - 20s 654ms/step - loss: 1.3175 - accuracy: 0.4742 - val_loss: 3.7063 - val_accuracy: 0.3292
Epoch 8/80

/Users/shalini/Documents/machine_learning_models/audio_classification_projects/myenv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


31/31 [==============================] - 21s 657ms/step - loss: 1.6116 - accuracy: 0.2955 - val_loss: 1.6562 - val_accuracy: 0.3333
Epoch 2/80
31/31 [==============================] - 21s 688ms/step - loss: 1.6265 - accuracy: 0.3326 - val_loss: 2.1019 - val_accuracy: 0.0864
Epoch 3/80
31/31 [==============================] - 20s 657ms/step - loss: 1.5542 - accuracy: 0.3605 - val_loss: 2.9656 - val_accuracy: 0.0864
Epoch 4/80
31/31 [==============================] - 20s 655ms/step - loss: 1.4684 - accuracy: 0.4008 - val_loss: 3.6099 - val_accuracy: 0.1646
Epoch 5/80
31/31 [==============================] - 21s 670ms/step - loss: 1.4098 - accuracy: 0.4246 - val_loss: 4.2826 - val_accuracy: 0.0864
Epoch 6/80
31/31 [==============================] - 21s 671ms/step - loss: 1.3769 - accuracy: 0.4659 - val_loss: 3.8791 - val_accuracy: 0.0864
Epoch 7/80
31/31 [==============================] - 21s 676ms/step - loss: 1.3868 - accuracy: 0.4370 - val_loss: 3.5672 - val_accuracy: 0.2922
Epoch 8/80

...

/Users/shalini/Documents/machine_learning_models/audio_classification_projects/myenv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


31/31 [==============================] - 21s 665ms/step - loss: 0.1488 - accuracy: 0.4267 - val_loss: 0.1647 - val_accuracy: 0.3292
Epoch 2/80
31/31 [==============================] - 21s 665ms/step - loss: 0.1396 - accuracy: 0.4804 - val_loss: 0.1733 - val_accuracy: 0.0864
Epoch 3/80
31/31 [==============================] - 20s 660ms/step - loss: 0.1397 - accuracy: 0.4597 - val_loss: 0.2683 - val_accuracy: 0.3292
Epoch 4/80
31/31 [==============================] - 20s 658ms/step - loss: 0.1373 - accuracy: 0.4814 - val_loss: 0.2659 - val_accuracy: 0.3292
Epoch 5/80
31/31 [==============================] - 20s 658ms/step - loss: 0.1315 - accuracy: 0.5093 - val_loss: 0.2660 - val_accuracy: 0.3292
Epoch 6/80
31/31 [==============================] - 20s 657ms/step - loss: 0.1305 - accuracy: 0.5186 - val_loss: 0.1527 - val_accuracy: 0.3292
Epoch 7/80
31/31 [==============================] - 20s 657ms/step - loss: 0.1301 - accuracy: 0.5052 - val_loss: 0.2159 - val_accuracy: 0.3292
Epoch 8/80

/Users/shalini/Documents/machine_learning_models/audio_classification_projects/myenv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


31/31 [==============================] - 21s 663ms/step - loss: 0.1550 - accuracy: 0.3058 - val_loss: 0.1562 - val_accuracy: 0.3292
Epoch 2/80
31/31 [==============================] - 20s 659ms/step - loss: 0.1409 - accuracy: 0.4236 - val_loss: 0.1542 - val_accuracy: 0.3292
Epoch 3/80
31/31 [==============================] - 21s 661ms/step - loss: 0.1384 - accuracy: 0.4649 - val_loss: 0.1536 - val_accuracy: 0.3292
Epoch 4/80
31/31 [==============================] - 20s 658ms/step - loss: 0.1344 - accuracy: 0.4824 - val_loss: 0.1544 - val_accuracy: 0.3292
Epoch 5/80
31/31 [==============================] - 20s 658ms/step - loss: 0.1342 - accuracy: 0.4928 - val_loss: 0.1571 - val_accuracy: 0.3292
Epoch 6/80
31/31 [==============================] - 20s 660ms/step - loss: 0.1328 - accuracy: 0.5083 - val_loss: 0.1615 - val_accuracy: 0.3292
Epoch 7/80
31/31 [==============================] - 20s 658ms/step - loss: 0.1291 - accuracy: 0.5238 - val_loss: 0.1672 - val_accuracy: 0.3292
Epoch 8/80

/Users/shalini/Documents/machine_learning_models/audio_classification_projects/myenv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
